# A/B-тестирование

Есть датасет с действиями пользователей, техническое задание и несколько вспомогательных датасетов.

**Цель исследования -** провести оценку результатов A/B-теста по тестированию изменений, связанных с внедрением улучшенной рекомендательной системы.

**Задачи исследования:**
1. Исследовать исходные данные.
2. Провести их преобработку.
3. Оценить корректность проведения теста, его соответствие ТЗ, изучить аудиторию теста и время его проведения.
4. Провести исследовательский анализ данных, выявить возможные особенности данных.
5. Сформулировать и проверить гипотезу об улучшении метрик контрольной группы.
6. На основе полученных результатов сделать вывод о проведенном А/В-тесте.

In [76]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np
import datetime as dt
import scipy.stats as stats
import math as mth
import plotly.express as px
from plotly import graph_objects as go
#import warnings
#warnings.filterwarnings('ignore')

## Обзор данных

In [77]:
events = pd.read_csv('final_ab_events.csv')
print(events.info())
events.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB
None


,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


In [78]:
marketing_events = pd.read_csv('ab_project_marketing_events.csv')
print(marketing_events.info())
marketing_events.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes
None


,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


In [79]:
new_users = pd.read_csv('final_ab_new_users.csv')
print(new_users.info())
new_users.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB
None


,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


In [80]:
participants = pd.read_csv('final_ab_participants.csv')
print(participants.info())
participants.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB
None


,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


Все столбцы с датами имеют неправильный тип данных, явных пропусков нигде нет, кроме столбца `details` в датасете с событиями.

## Предобработка

In [81]:
# приведем столбцы с датами к соответствующему типу данных
events['event_dt'] = pd.to_datetime(events['event_dt'])
print("Тип events['event_dt'] -", events['event_dt'].dtype)

marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])
print("\nТип marketing_events['start_dt'] -", marketing_events['start_dt'].dtype)
print("Тип marketing_events['finish_dt'] -", marketing_events['finish_dt'].dtype)

new_users['first_date'] = pd.to_datetime(new_users['first_date'])
print("\nТип new_users['first_date'] -", new_users['first_date'].dtype)

Тип events['event_dt'] - datetime64[ns]

Тип marketing_events['start_dt'] - datetime64[ns]
Тип marketing_events['finish_dt'] - datetime64[ns]

Тип new_users['first_date'] - datetime64[ns]


In [82]:
events.loc[events['details'].isna(), 'event_name'].unique()

array(['product_cart', 'product_page', 'login'], dtype=object)

Пропуски есть только в столбце с деталями к событиям, но приходятся они на все события кроме purchase, для которых там хранится стоимость покупки.

## Оценка корректности проведения теста

### Соответствие данных требованиям технического задания

Проверим корректность всех пунктов технического задания:

- Название теста: recommender_system_test

In [83]:
participants['ab_test'].unique()

array(['recommender_system_test', 'interface_eu_test'], dtype=object)

В данных присутствует информация о двух проведенных тестах. Посмотрим сколько пользователей попало в каждый из них, и сколько попало в оба.

In [84]:
rst = participants.loc[participants['ab_test'] == 'recommender_system_test', 'user_id']
print('Участвовало в recommender_system_test', len(rst))

Участвовало в recommender_system_test 6701


In [85]:
iet = participants.loc[participants['ab_test'] == 'interface_eu_test', 'user_id']
print('Участвовало в interface_eu_test', len(iet))

Участвовало в interface_eu_test 11567


In [86]:
rst_and_iet = list(set(rst) & set(iet))
print('Участвовало в обоих тестах', len(rst_and_iet))

Участвовало в обоих тестах 1602


- группы: А — контрольная, B — новая платёжная воронка

In [87]:
participants.loc[participants['ab_test'] == 'recommender_system_test', 'group'].value_counts()

A    3824
B    2877
Name: group, dtype: int64

- даты:
    - дата запуска: 2020-12-07;
    - дата остановки набора новых пользователей: 2020-12-21;
    - дата остановки: 2021-01-04

In [88]:
new_users_rst = new_users.loc[new_users['user_id'].isin(rst)]
events_rst = events.loc[events['user_id'].isin(rst)]

print(f"Первая дата регистрации нового пользователя участвовавшего в тесте recommender_system_test:\n\
{new_users_rst['first_date'].min()}\n")

print(f"Последняя дата регистрации нового пользователя участвовавшего в тесте recommender_system_test:\n\
{new_users_rst['first_date'].max()}\n")

print(f"Первое действие пользователя участвовавшего в тесте recommender_system_test:\n\
{events_rst['event_dt'].min()}\n")

print(f"Последнее действие пользователя участвовавшего в тесте recommender_system_test:\n\
{events_rst['event_dt'].max()}")

Первая дата регистрации нового пользователя участвовавшего в тесте recommender_system_test:
2020-12-07 00:00:00

Последняя дата регистрации нового пользователя участвовавшего в тесте recommender_system_test:
2020-12-21 00:00:00

Первое действие пользователя участвовавшего в тесте recommender_system_test:
2020-12-07 00:05:57

Последнее действие пользователя участвовавшего в тесте recommender_system_test:
2020-12-30 12:42:57


- аудитория: 15% новых пользователей из региона EU

In [89]:
new_users_rst_eu = new_users_rst.query('region == "EU"')
len(new_users_rst_eu) / len(new_users.query('region == "EU" & first_date < datetime(2020, 12, 22)')) * 100

15.0

- ожидаемое количество участников теста: 6000

In [90]:
len(new_users_rst_eu)

6351

- ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
    - конверсии в просмотр карточек товаров — событие `product_page`,
    - просмотры корзины — `product_cart`,
    - покупки — `purchase`.

In [91]:
# для начала построит лайфтаймы пользователей и очистим датасет от тех событий что вышли за порог в 14 дней
ab_test = new_users_rst_eu.merge(participants.loc[participants['ab_test'] == 'recommender_system_test'], on='user_id', how='left')\
.merge(events_rst, on='user_id', how='left')
ab_test['days'] = (ab_test['event_dt'] - ab_test['first_date']).dt.days
ab_test['event_date'] = ab_test['event_dt'].dt.date
ab_test.query('days >= 14')

,user_id,first_date,region,device,group,ab_test,event_dt,event_name,details,days,event_date
8,DD4352CDCF8C3D57,2020-12-07,EU,Android,B,recommender_system_test,2020-12-30 12:42:57,product_page,NaN,23.0,2020-12-30
14,DD4352CDCF8C3D57,2020-12-07,EU,Android,B,recommender_system_test,2020-12-30 12:42:56,login,NaN,23.0,2020-12-30
94,75845C83258FBF73,2020-12-07,EU,Android,B,recommender_system_test,2020-12-30 06:42:52,product_cart,NaN,23.0,2020-12-30
98,75845C83258FBF73,2020-12-07,EU,Android,B,recommender_system_test,2020-12-30 06:42:52,login,NaN,23.0,2020-12-30
110,6A581C74EF4D5F44,2020-12-07,EU,iPhone,B,recommender_system_test,2020-12-29 21:46:48,login,NaN,22.0,2020-12-29
...,...,...,...,...,...,...,...,...,...,...,...
23948,7DC0F01827866E41,2020-12-13,EU,iPhone,A,recommender_system_test,2020-12-29 08:43:44,product_page,NaN,16.0,2020-12-29
23950,7DC0F01827866E41,2020-12-13,EU,iPhone,A,recommender_system_test,2020-12-29 08:43:44,login,NaN,16.0,2020-12-29
24047,48BEDF1889241425,2020-12-13,EU,Mac,A,recommender_system_test,2020-12-28 20:26:13,purchase,4.99,15.0,2020-12-28
24051,48BEDF1889241425,2020-12-13,EU,Mac,A,recommender_system_test,2020-12-28 20:26:16,product_page,NaN,15.0,2020-12-28


800 событий происходят позже чем 14 дней с момента регистрации пользователей, что не подходит под условие ТЗ, поэтому удалим их, но сперва посмотрим как события распределяются по группам.

In [92]:
fig = px.histogram(ab_test, x='days', color='group')
fig.update_layout(title='Распределение событий по лайфтаймам в разрезе групп',
                   xaxis_title='Лайфтайм',
                   yaxis_title='Количество событий')
fig.show()

Как видно по графику событий у группы В намного меньше чем у А, так же видно что наибольшее количество событий пользователи совершают в нулевой лайфтайм - день регистрации, а после 15 лайфтайма количество событий становится меньше 100 в день.

In [93]:
# удалим события после 14ого дня с момента регистрации
ab_test = ab_test.query('days < 14 or event_dt.isna()')

Так же после объединения датасетов у нас появились пропуски, то есть были пользователи которые после регистрации не совершали никаких действий. Рассмотрим их.

In [94]:
print(f"{ab_test.loc[ab_test['event_dt'].isna(), 'user_id'].nunique()} пользователей \
после регистрации не совершали никаких действий,\nчто составляет \
{ab_test.loc[ab_test['event_dt'].isna(), 'user_id'].nunique()/ab_test['user_id'].nunique():.2%} \
от общего числа пользователей участвовавших в тесте")

print(f"В группе А оказалось \
{ab_test.loc[(ab_test['event_dt'].isna()) & (ab_test['group'] == 'A'), 'user_id'].nunique()/ab_test.loc[ab_test['group'] == 'A', 'user_id'].nunique():.2%} неактивных пользователей")

print(f"В группе B оказалось \
{ab_test.loc[(ab_test['event_dt'].isna()) & (ab_test['group'] == 'B'), 'user_id'].nunique()/ab_test.loc[ab_test['group'] == 'B', 'user_id'].nunique():.2%} неактивных пользователей")

2870 пользователей после регистрации не совершали никаких действий,
что составляет 45.19% от общего числа пользователей участвовавших в тесте
В группе А оказалось 28.34% неактивных пользователей
В группе B оказалось 67.72% неактивных пользователей


Полученный процент неактивных пользователей очень велик, что может существенно снизить мощность теста и  повлиять на его результаты.

In [95]:
# рассмотрим как формировались группы пользователей
groups = ab_test.groupby(['first_date', 'group'], as_index=False).agg({'user_id': 'nunique'})
fig = px.line(groups, x='first_date', y='user_id', color='group')
fig.update_layout(title='Формирование групп теста',
                   xaxis_title='Дата регистрации',
                   yaxis_title='Количество пользователей')
fig.show()

В группу В всегда попадало меньше пользователей, при этом линии повторяют изгибы друг друга.

In [96]:
inactive_users = ab_test.query('event_dt.isna()').groupby(['first_date', 'group'], as_index=False).agg({'user_id': 'count'})
fig = px.line(inactive_users, x='first_date', y='user_id', color='group')
fig.update_layout(title='Неактивные пользователи в разрезе групп',
                   xaxis_title='Дата регистрации',
                   yaxis_title='Количество неактивных пользователей')
fig.show()

Как видно из графика пользователи обеих групп зарегистрированные с 7ого по 13ое декабря в равной степени становились неактивными, тогда как у зарегистрированных с 14ого декабря у группы А неактивных пользователей не стало. Это явно указывает на ошибку в данных или некорректное проведение теста с 14 декабря.

In [97]:
# теперь удалим неактивных пользователей
ab_test = ab_test.query('not event_dt.isna()')
ab_test['user_id'].nunique()

3481

После удаления неактивных пользователей, у нас осталось их почти в 2 раза меньше, что нарушает условие ТЗ об ожидаемом количестве в 6000 и снижает чувствительность теста.

In [98]:
# построим воронки для каждой группы
ab_test_A = ab_test.query('group == "A"')
ab_test_B = ab_test.query('group == "B"')
funnel = ['login', 'product_page', 'product_cart', 'purchase']
funnel_A = ab_test_A.groupby('event_name').agg({'user_id': 'nunique'}).reindex(funnel).reset_index()
funnel_A['share'] = funnel_A['user_id'] / funnel_A['user_id'].shift(1, fill_value=2604)
funnel_A.style.format({'share': '{:.2%}'})

,event_name,user_id,share
0,login,2604,100.00%
1,product_page,1685,64.71%
2,product_cart,782,46.41%
3,purchase,833,106.52%


In [99]:
funnel_B = ab_test_B.groupby('event_name').agg({'user_id': 'nunique'}).reindex(funnel).reset_index()
funnel_B['share'] = funnel_B['user_id'] / funnel_B['user_id'].shift(1, fill_value=876)
funnel_B.style.format({'share': '{:.2%}'})

,event_name,user_id,share
0,login,876,100.00%
1,product_page,493,56.28%
2,product_cart,244,49.49%
3,purchase,249,102.05%


In [100]:
round((funnel_B['share'] / funnel_A['share'] - 1) * 100, 2)

0     0.00
1   -13.03
2     6.64
3    -4.20
Name: share, dtype: float64

Не все условия ТЗ соблюдены корректно, например есть пользователи которые участвовали в двух тестах, фактическая дата остановки теста `2020-12-30` не соответсвует согласованной `2021-01-04`, не выполняется условие с улучшением каждой метрики не менее, чем на 10%. Проверим датасеты по следующим пунктам:

### Аудитория теста

In [101]:
# проверим пересечения с конкурирующим тестом 
rst_and_iet_new = ab_test.query('user_id in @rst_and_iet')['user_id'].unique()
len(rst_and_iet_new)

887

В нашем тесте участвуют 887 пользователей, которые так же участвовал во втором тесте. Расмотрим как они распределяются по группам.

In [102]:
iet_group_A = participants.query('ab_test == "interface_eu_test" & user_id in @rst_and_iet_new & group == "A"')['user_id']
iet_group_B = participants.query('ab_test == "interface_eu_test" & user_id in @rst_and_iet_new & group == "B"')['user_id']
rst_group_A = participants.query('ab_test == "recommender_system_test" & user_id in @rst_and_iet_new & group == "A"')['user_id']
rst_group_B = participants.query('ab_test == "recommender_system_test" & user_id in @rst_and_iet_new & group == "B"')['user_id']
print(f'В группе А recommender_system_test участвовало:\n\
{len(list(set(iet_group_A) & set(rst_group_A)))} пользователей из группы А теста interface_eu_test\n\
{len(list(set(iet_group_B) & set(rst_group_A)))} из группы В\n')
print(f'В группе В recommender_system_test участвовало:\n\
{len(list(set(iet_group_A) & set(rst_group_B)))} пользователей из группы А теста interface_eu_test\n\
{len(list(set(iet_group_B) & set(rst_group_B)))} из группы В')


В группе А recommender_system_test участвовало:
340 пользователей из группы А теста interface_eu_test
325 из группы В

В группе В recommender_system_test участвовало:
116 пользователей из группы А теста interface_eu_test
106 из группы В


Так как мы не знаем дизайна второго теста, а пользователи из второго теста разных групп распределены в наших группах примерно одинаково и могут нивелировать действия друг друга в случае сильного влияния второго теста, соотношение конверсий группы А относительно группы В это не должно измениться, поэтому оставим тех пользователей, которые участвовали в обоих тестах во избежание снижения мощности теста.

In [103]:
# проверим наличие пользователей, участвующих в двух группах теста одновременно
ab_test.groupby('user_id').agg({'group': 'nunique'}).query('group > 1').count()

group    0
dtype: int64

In [104]:
# проверим равномерность распределения по тестовым группам и правильность их формирования
groups_new = ab_test.groupby('group', as_index=False).agg({'user_id': 'nunique'})
groups_new['share'] = groups_new['user_id'] / ab_test['user_id'].nunique()
groups_new.style.format({'share': '{:.2%}'})

,group,user_id,share
0,A,2604,74.81%
1,B,877,25.19%


Такое распределение пользователей по группам не должно повлиять на изменение конверсии, но тем не менее уменьшается чувствительность теста.

### Время проведения теста

In [105]:
# убедимся, что оно не совпадает с маркетинговыми и другими активностями
marketing_events.query('regions.str.contains("EU") & finish_dt > datetime(2020, 12, 7)')

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03


Во время проведения теста проходила одна маркетинговая акция - рождественская. Необходимо посмотреть сказалась ли она на действиях пользователей.

In [106]:
# рассмотрим распределение событий пользователей по времени
fig = px.histogram(ab_test, x='event_date')
fig.update_layout(title='Распределение событий по дате',
                   xaxis_title='Дата события',
                   yaxis_title='Количество событий')
fig.show()

Несмотря на начало маркетинговой акции 25 декабря, активность пользователей всё равно падала начиная с 21ого, поэтому будем считать, что рождественская кампания не сильно отразилась на результатах теста.

## Исследовательский анализ

### Количество событий на пользователя одинаково распределены в выборках?

In [107]:
events_per_users = ab_test.groupby(['group', 'user_id'], as_index=False)\
                            .agg({'event_name': 'count'}).rename(columns={'event_name': 'events_count'})
print('Среднее количество событий на пользователя в группе А:',\
      round(events_per_users.query('group == "A"')['events_count'].mean(), 2))
print('Среднее количество событий на пользователя в группе B:',\
      round(events_per_users.query('group == "B"')['events_count'].mean(), 2))

Среднее количество событий на пользователя в группе А: 6.85
Среднее количество событий на пользователя в группе B: 5.46


In [108]:
events_amount = events_per_users.groupby(['group', 'events_count'], as_index=False).agg({'user_id': 'count'})
fig = px.bar(events_amount, x='events_count', y='user_id', color='group')
fig.update_xaxes(dtick=1)
fig.update_layout(title='Количество событий, совершенных пользователями',
                   xaxis_title='Количество событий',
                   yaxis_title='Количество пользователей')
fig.show()

Больше всего пользователей обеих групп совершило 6 или 4 события, больше 20 событий совершило всего 4 пользователя. Группа А совершила большее не только абсолютное количество событий, но и по среднему значению также лидирует.

### Как число событий в выборках распределено по дням?

In [109]:
events_per_days = ab_test.groupby(['group', 'event_date'], as_index=False).agg({'event_name': 'count'})
fig = px.bar(events_per_days, x='event_date', y='event_name', color='group')
fig.update_layout(title='Количество событий по дням',
                   xaxis_title='Дата',
                   yaxis_title='Количество событий')
fig.show()

Распределение количества событий по дням не равномерное: в первую неделю нет всплесков, но во вторую виден резкий скачок группы А, с пиковым значением у обоих групп 21ого декабря, в третью неделю количество событий стало уменьшаться.

### Как меняется конверсия в воронке в выборках на разных этапах?

In [110]:
funnel = ['login', 'product_page', 'product_cart', 'purchase']

In [111]:
funnel_per_group = ab_test.groupby(['group', 'event_name'])\
                        .agg({'user_id': 'nunique'}).reindex(funnel, level=1).reset_index()

fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'A',
    y = funnel_per_group.loc[funnel_per_group['group'] == 'A', 'event_name'],
    x = funnel_per_group.loc[funnel_per_group['group'] == 'A', 'user_id'],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'B',
    y = funnel_per_group.loc[funnel_per_group['group'] == 'B', 'event_name'],
    x = funnel_per_group.loc[funnel_per_group['group'] == 'B', 'user_id'],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.update_layout(title='Воронка событий')

fig.show()

В группе В на всех этапах конверия в принципе ниже чем в группе А. В группе А до покупки доходит 32% пользователей, в группе В - 28%.

### Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?

Важным моментом является то, что воронка не совсем последовательна, так как покупку можно совершить минуя корзину, это может повлиять на результаты теста. Также стоит обратить внимание на то, что группы распределены неравномерно, тест проводится в сезон пользовательской активности (католическое рождество и преддверие нового года), и в целом данные по многим пунктам не соответствуют ТЗ, что вероятно может исказить результаты.

## Проверка гипотез

По результатам тестирования группа В не показала улучшения метрик на ожидаемые 10%, скорее наоборот, изменение рекомендательной системы привело к ухудшению конверсии пользователей в покупатели. Оценим статистическую значимость различия долей конверсии с помощью z-критерия.

Сформулируем гипотезы:<br>
**Нулевая (Н0)**: доли уникальных пользователей, побывавших на этапе воронки, одинаковы<br>
**Альтернативная (Н1)**: между долями уникальных посетителей, побывавших на этапе воронки, есть значимая разница.

Так как тесты множественные, т.е. проводятся несколько раз на одних и тех же данных, используем метод корректировки уровня значимости - поправку Бонферрони.

In [112]:
alpha = 0.05 / 4
alpha

0.0125

In [113]:
# напишем функцию для автоматизации теста
def test_z(group1, group2, event, alpha):

    def successes_trials(group1, group2, event):
    
        successes = np.array([ab_test.query(f'group == "{group1}" & event_name == "{event}"')['user_id'].nunique(),\
                          ab_test.query(f'group == "{group2}" & event_name == "{event}"')['user_id'].nunique()])

        trials = np.array([ab_test.query(f'group == "{group1}"')['user_id'].nunique(),\
                           ab_test.query(f'group == "{group2}"')['user_id'].nunique()])
        return successes, trials
    
    successes, trials = successes_trials(group1, group2, event) 
    # пропорция успехов в первой группе:
    p1 = successes[0]/trials[0]
 
    # пропорция успехов во второй группе:
    p2 = successes[1]/trials[1]
 
    # пропорция успехов в комбинированном датасете:
    p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])
 
    # разница пропорций в датасетах
    difference = p1 - p2 
 
    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))
 
    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = stats.norm(0, 1) 
 
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    print(f'Результаты теста для групп {group1} и {group2} и события {event}')
 
    print('p-значение: ', p_value)
    if p_value < alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')

In [114]:
test_z('A', 'B', 'login', alpha)

Результаты теста для групп A и B и события login
p-значение:  0.08481837035887363
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [115]:
test_z('A', 'B', 'product_page', alpha)

Результаты теста для групп A и B и события product_page
p-значение:  6.942739359416805e-06
Отвергаем нулевую гипотезу: между долями есть значимая разница


In [116]:
test_z('A', 'B', 'product_cart', alpha)

Результаты теста для групп A и B и события product_cart
p-значение:  0.21469192029582396
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [117]:
test_z('A', 'B', 'purchase', alpha)

Результаты теста для групп A и B и события purchase
p-значение:  0.04652482738393027
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


По результатам стат.тестов в 3 из 4 этапах воронки доли пользователей оказались одинаковы, а на этапе просмотра карточки товара выявлена статистически значимая разница конверсий, то есть процент пользователей группы В перешедших на карточку товара действительно снизился после внедрения новой рекомендательной системы. Таким образом эксперимент можно считать неудачным. 

## Вывод

В ходе проведенного анализа результатов A/B-теста за период 07.12.2020 по 30.12.2020 было выявлено, что данные во многом не соответствуют ТЗ, тест проводится в сезон праздников, группы набраны неравномерно и в целом можно сказать что тест проведен некорректно.<br>
Также тест не показал статистически значимого изменения ключевых метрик, в то время как ожидалось +10%, а на одном из этапов воронки показал наоборот значимое уменьшение.<br>
Таким образом мы можем принять результаты теста и рекомендовать не использовать новую систему рекомендаций, как не соответствующую ожиданиям, но с поправкой на нарушения в части проведения теста, которые могли критично исказить полученные результаты.